# Pruebas Filtro:

- Lenguaje Filtro (Python)
- Crear sistema de web scraping, y crear dataset de al menos 5 categorías , con datos obtenidos a       través del web scraping.
- Las categorías deben de estar numéricamente balanceadas en cantidad.
- Se debe crear un procedimiento automatizado de preprocesamiento .
- Se debe desarrollar un modelo de redes neuronales artificiales con tensor flow/keras , para el       procesamiento de este dataset.
- Se debe crear una api rest , que proporcione por medio de métodos http , un ingreso de datos y una   salida de la clasificación del mismo.

# Clases para periodismo

- Política y economía
- Ciencia, tecnología, salud, cultura, innovación y emprendedorismo
- Viajes, turismo
- Espectáculos, recreación, gastronomía y sociales
- Deportes

**Vamos a hacer que cada línea del dataset sea un párrafo de la web scrapeada. Vamos a obtener una nueva lista de palabras confusas y entrenar el modelo luego de haberlas excluido**

In [1]:
# La idea es transformar estos programas en un paquete de bibliotecas utilizables por un
# usuario nivel Excel avanzado. Todos los valores hardcodeados en esta celda serán 
# parámetros configurables de la biblioteca a generar

clases = 5
columna = 3

In [2]:
from requests import get # la vamos a usar para solicitar la página mediante su URL 
from bs4 import BeautifulSoup # biblioteca para web scraping
import numpy as np
import pandas as pd
import string                              
from nltk.corpus import stopwords # para remover palabras que no agregan significado          
from nltk.stem import PorterStemmer # para llevar las palabras a su forma raíz        
from nltk.tokenize import TweetTokenizer # para transformar cada palabra en un token
from IPython.display import clear_output
from collections import defaultdict
import nltk
import pickle
import random

In [3]:
def save_obj(obj, name):
       with open(name + '.pkl', 'wb') as f:
            pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
       with open(name + '.pkl', 'rb') as f:
            return pickle.load(f)

In [26]:
def limpiarTextoCrudo(st):
    res = []
    for t in range(500):
        st = st.replace("[" + str(t) + "]", " ")
    for letra in ["a","b","d","e","f","g","h","i","j"]:
        st = st.replace("[" + letra + "]", " ")
    caracteresARemover = ["(",")","[","]",".",",",'"',"“",":"]
    for signo in caracteresARemover:
        st = st.replace(signo," ")
    st = st.split()
    hizoJoin = False
    for i in range(len(st) - 1):
        if st[i].isdecimal() and i < len(st) -1 and (
                                   st[i+1].lower() == "ad" or 
                                   st[i+1].lower() == "bc" or 
                                   st[i+1].lower() == "ac" or 
                                   st[i+1].lower() == "dc"):
            res.append("".join([st[i],st[i+1]]))
            hizoJoin = True 
        elif st[i].lower() in {"ad","bc","ac","dc"} and hizoJoin:
            hizoJoin = False    
        elif st[i].lower() not in {"ad","bc","ac","dc"} or not hizoJoin:
            res.append(st[i])
            hizoJoin = False        
    if st[-1].lower() not in {"ad","bc","ac","dc"}: 
        res.append(st[-1])
    return " ".join(res)

In [6]:
parrafo = "Alexander III of Macedon (Greek: Αλέξανδρος Γʹ ὁ Μακεδών, Aléxandros III ho Makedȏn; 20/21 July 356 BC – 10/11 June 323 BC), commonly known as Alexander the Great (Greek: ὁ Μέγας, ho Mégas), was a king (basileus) of the ancient Greek kingdom of Macedon[a] and a member of the Argead dynasty. He was born in Pella in 356 BC and succeeded his father Philip II to the throne at the age of 20. He spent most of his ruling years on an unprecedented military campaign through western Asia and northeast Africa, and by the age of thirty, he had created one of the largest empires of the ancient world, stretching from Greece to northwestern India.[1][2] He was undefeated in battle and is widely considered one of history's most successful military commanders.[3]\n"
limpiarTextoCrudo(parrafo)

"Alexander III of Macedon Greek: Αλέξανδρος Γʹ ὁ Μακεδών Aléxandros III ho Makedȏn; 20/21 July 356BC – 10/11 June 323BC commonly known as Alexander the Great Greek: ὁ Μέγας ho Mégas was a king basileus of the ancient Greek kingdom of Macedon and a member of the Argead dynasty He was born in Pella in 356BC and succeeded his father Philip II to the throne at the age of 20 He spent most of his ruling years on an unprecedented military campaign through western Asia and northeast Africa and by the age of thirty he had created one of the largest empires of the ancient world stretching from Greece to northwestern India He was undefeated in battle and is widely considered one of history's most successful military commanders"

In [ ]:
# Esta clase no la vamos a usar ahora, le mejoraremos en futuras notebooks
class Textos:
    
    def __init__(self, clases, webs):
        self.clases = clases
        self.lista_dataset = []
        self.crear_dataframe(clases, webs)

    def crear_dataframe(self, clases, webs):
        self.lista_dataset.append('TextoCrudo')
        self.lista_dataset.append('TextoLimpio')

        for i in range(1, clases + 1):
            self.lista_dataset.append('Clase' + str(i))

        self.lista_dataset.append('Predicción')
        self.lista_dataset.append('CLASE')
        self.dataframe = pd.DataFrame(columns=self.lista_dataset)
        self.webs = pd.ExcelFile(webs)
        df = self.webs.parse(self.webs.sheet_names[0])
        self.__clases_df = set(df["CÓDIGO"].value_counts().index)

        for i in df.index:
            print(f"Creando Dataframe {i+1} de {len(df.index)}")
            url = df["URL"][i]
            req = get(url)
            statusCode = req.status_code
            html = BeautifulSoup(req.text, "html.parser")

            if statusCode == 200:
                paragraphs = html.select("p")
                cont = 0

                for para in paragraphs:
                    if len(para.text) < 15:
                        continue
                    cont += 1
                    self.dataframe = self.dataframe.append({'TextoCrudo': para.text, 'CLASE': df["CÓDIGO"][i]}, ignore_index=True)

                clear_output(wait=True)

    def limpiar(textoCrudo):
        for i in range(100):
            textoCrudo = textoCrudo.replace("[" + str(i) + "]", "")
        
        for i in ["a","b","c"]:
            textoCrudo = textoCrudo.replace("[" + i + "]", "")
        
        stopwords_english = stopwords.words('english')
        stop        = stopwords.words('english')
        tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
        stemmer = PorterStemmer()    
        texto_tokens = tokenizer.tokenize(textoCrudo)
        sufijos_fecha = ["bc","ac"]

        for i in range(len(texto_tokens.copy())):
            if i > len(texto_tokens) - 1:
                break
            if i !=0:
                if texto_tokens[i] in sufijos_fecha and texto_tokens[i - 1].isnumeric():
                    texto_tokens[i] = texto_tokens[i - 1] + texto_tokens[i]     
                    texto_tokens.pop(i - 1)
                           
        texto_clean  = [ word for word in texto_tokens if word not in stop and word not in string.punctuation]  
        texto_stem   = [stemmer.stem(word) for word in texto_clean] 
        return texto_stem       
      

In [7]:
lista_dataset = []
lista_dataset.append('TextoCrudo')
lista_dataset.append('TextoLimpio')
lista_dataset.append('TextoProcesado')
for i in range(1, clases + 1):
    lista_dataset.append('Clase' + str(i))
lista_dataset.append('MaxFreq')
lista_dataset.append('Predicción')
lista_dataset.append('CLASE')
dataFrame = pd.DataFrame(columns = lista_dataset)

In [8]:
textos = dataFrame

In [9]:
textos

,TextoCrudo,TextoLimpio,TextoProcesado,Clase1,Clase2,Clase3,Clase4,Clase5,MaxFreq,Predicción,CLASE


In [10]:
# Webs.xls es una planilla de conteniendo URLs y la clase a la que corresponde 
# Cada usuario debería crear su propia planilla
# TODO: interfaz para crear la planilla
webs = pd.ExcelFile("Webs.xls")

In [11]:
df = webs.parse(webs.sheet_names[0])
clases_df = set(df["CÓDIGO"].value_counts().index)
clases_df

{1, 2, 3, 4, 5}

In [13]:
df.sample(10)

,URL,NOMBRE ARCHIVO,CLASE,CÓDIGO
34,https://en.wikipedia.org/wiki/Henry_VIII,Henry_VIII,EDAD MODERNA,3
40,https://en.wikipedia.org/wiki/https://en.wikip...,Louis_XV,EDAD MODERNA,3
51,https://en.wikipedia.org/wiki/French_Revolution,French_Revolution,EDAD CONTEMPORÁNEA,4
49,https://en.wikipedia.org/wiki/Renaissance,Renaissance,EDAD MODERNA,3
73,https://en.wikipedia.org/wiki/Maglemosian_culture,Maglemosian_culture,PREHISTORIA,5
37,https://en.wikipedia.org/wiki/Louis_XIV,Louis_XIV,EDAD MODERNA,3
8,https://en.wikipedia.org/wiki/Sumer,Sumer,EDAD ANTIGUA,1
14,https://en.wikipedia.org/wiki/Early_Middle_Ages,Early_Middle_Ages,EDAD MEDIA,2
4,https://en.wikipedia.org/wiki/Julius_Caesar,Julius_Caesar,EDAD ANTIGUA,1
27,https://en.wikipedia.org/wiki/Byzantine_Empire,Byzantine_Empire,EDAD MEDIA,2


In [14]:
for i in df.index:
    print(f"Creando Dataframe {i+1} de {len(df.index)}")
    url = df["URL"][i]
    req = get(url)
    statusCode = req.status_code
    if statusCode == 200:
        html = BeautifulSoup(req.text, "html.parser")
        paragraphs = html.select("p")
        cont = 0
        for para in paragraphs:
            if len(para.text) < 40:
                continue
            cont += 1
            dataFrame = dataFrame.append({'TextoCrudo': para.text, 'CLASE': df["CÓDIGO"][i]}, ignore_index=True)
    clear_output(wait=True)
dataFrame

,TextoCrudo,TextoLimpio,TextoProcesado,Clase1,Clase2,Clase3,Clase4,Clase5,MaxFreq,Predicción,CLASE
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,"During his youth, Alexander was tutored by Ari...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,"Alexander endeavoured to reach the ""ends of th...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,Alexander's legacy includes the cultural diffu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,"Alexander was born in Pella, the capital of th...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...
5937,There are nevertheless a number of contemporar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
5938,Evidence suggests big-game hunter-gatherers cr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
5939,Hunter-gatherers would eventually flourish all...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
5940,The Archaic period in the Americas saw a chang...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [15]:
textos = dataFrame
textos.sample(10) 

,TextoCrudo,TextoLimpio,TextoProcesado,Clase1,Clase2,Clase3,Clase4,Clase5,MaxFreq,Predicción,CLASE
1640,Charlemagne used these circumstances to claim ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3327,"On 31 March, another declaration—sent by Grand...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4344,Communist and fascist movements around Europe ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
4944,According to recent studies [28] the BMAC was ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
3917,British diplomacy also played a key role; in p...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
1459,The independent state of Salerno inspired the ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
5778,"""They [...] developed steepling thought-struct...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
1426,"In 47, a struggle ensued amongst the Cherusci ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2651,By the end of the Scientific Revolution the qu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
5075,The earliest documented representative of the ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [16]:
textos.to_csv("HistoriaTextosCrudos.csv")

In [17]:
textos = pd.read_csv("HistoriaTextosCrudos.csv")
textos = textos.iloc[:,1:]
textos.head(10)

,TextoCrudo,TextoLimpio,TextoProcesado,Clase1,Clase2,Clase3,Clase4,Clase5,MaxFreq,Predicción,CLASE
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,"During his youth, Alexander was tutored by Ari...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,"Alexander endeavoured to reach the ""ends of th...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,Alexander's legacy includes the cultural diffu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,"Alexander was born in Pella, the capital of th...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
5,Several legends surround Alexander's birth and...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
6,"On the day Alexander was born, Philip was prep...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
7,"In his early years, Alexander was raised by a ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
8,"When Alexander was ten years old, a trader fro...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9,"When Alexander was 13, Philip began to search ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [18]:
textos["TextoCrudo"][5]

'Several legends surround Alexander\'s birth and childhood.[12] According to the ancient Greek biographer Plutarch, on the eve of the consummation of her marriage to Philip, Olympias dreamed that her womb was struck by a thunderbolt that caused a flame to spread "far and wide" before dying away. Sometime after the wedding, Philip is said to have seen himself, in a dream, securing his wife\'s womb with a seal engraved with a lion\'s image.[13] Plutarch offered a variety of interpretations of these dreams: that Olympias was pregnant before her marriage, indicated by the sealing of her womb; or that Alexander\'s father was Zeus. Ancient commentators were divided about whether the ambitious Olympias promulgated the story of Alexander\'s divine parentage, variously claiming that she had told Alexander, or that she dismissed the suggestion as impious.[13]\n'

In [19]:
textos.groupby(by=["CLASE"]).count()

,TextoCrudo,TextoLimpio,TextoProcesado,Clase1,Clase2,Clase3,Clase4,Clase5,MaxFreq,Predicción
CLASE,,,,,,,,,,
1,1182,0,0,0,0,0,0,0,0,0
2,1285,0,0,0,0,0,0,0,0,0
3,1228,0,0,0,0,0,0,0,0,0
4,1034,0,0,0,0,0,0,0,0,0
5,1213,0,0,0,0,0,0,0,0,0


In [20]:
for i in textos.index:
    clear_output(wait=True)        
    print(f"Limpiando Texto Crudo {i+1} de {len(textos.index)}")
    textos["TextoLimpio"][i] = limpiarTextoCrudo(textos["TextoCrudo"][i])
textos

Limpiando Texto Crudo 5942 de 5942


,TextoCrudo,TextoLimpio,TextoProcesado,Clase1,Clase2,Clase3,Clase4,Clase5,MaxFreq,Predicción,CLASE
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,Alexander III of Macedon Greek: Αλέξανδρος Γʹ ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,"During his youth, Alexander was tutored by Ari...",During his youth Alexander was tutored by Aris...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,"Alexander endeavoured to reach the ""ends of th...",Alexander endeavoured to reach the ends of the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,Alexander's legacy includes the cultural diffu...,Alexander's legacy includes the cultural diffu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,"Alexander was born in Pella, the capital of th...",Alexander was born in Pella the capital of the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...
5937,There are nevertheless a number of contemporar...,There are nevertheless a number of contemporar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
5938,Evidence suggests big-game hunter-gatherers cr...,Evidence suggests big-game hunter-gatherers cr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
5939,Hunter-gatherers would eventually flourish all...,Hunter-gatherers would eventually flourish all...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
5940,The Archaic period in the Americas saw a chang...,The Archaic period in the Americas saw a chang...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [22]:
i = 4
print()
print()
print(textos["TextoCrudo"][i])
print()
print(textos["TextoLimpio"][i])



Alexander was born in Pella, the capital of the Kingdom of Macedon,[8] on the sixth day of the ancient Greek month of Hekatombaion, which probably corresponds to 20 July 356 BC, although the exact date is uncertain.[9] He was the son of the king of Macedon, Philip II, and his fourth wife, Olympias, the daughter of Neoptolemus I, king of Epirus.[10] Although Philip had seven or eight wives, Olympias was his principal wife for some time, likely because she gave birth to Alexander.[11]


Alexander was born in Pella the capital of the Kingdom of Macedon on the sixth day of the ancient Greek month of Hekatombaion which probably corresponds to 20 July 356BC although the exact date is uncertain He was the son of the king of Macedon Philip II and his fourth wife Olympias the daughter of Neoptolemus I king of Epirus Although Philip had seven or eight wives Olympias was his principal wife for some time likely because she gave birth to Alexander


In [23]:
textos.sample(10)

,TextoCrudo,TextoLimpio,TextoProcesado,Clase1,Clase2,Clase3,Clase4,Clase5,MaxFreq,Predicción,CLASE
3820,The most heated controversy was over the statu...,The most heated controversy was over the statu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
4275,The widespread use of chemical warfare was a d...,The widespread use of chemical warfare was a d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
3717,"On 10 June, Sieyès moved that the Third Estate...",On 10 June Sieyès moved that the Third Estate ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
2080,The majority of runic inscriptions from the Vi...,The majority of runic inscriptions from the Vi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2276,"Between 850 and 1100, the Empire developed a m...",Between 850 and 1100 the Empire developed a mi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
5461,Times from the 8000 BCE to about 3000 BCE may ...,Times from the 8000 BCE to about 3000 BCE may ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
5904,A hunter-gatherer is a nomadic[1] human living...,A hunter-gatherer is a nomadic human living in...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
4902,Some species of non-primates are able to use s...,Some species of non-primates are able to use s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
2641,\nCan well direct him where to look for it.[15],Can well direct him where to look for it,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
2696,Various other advances in medical understandin...,Various other advances in medical understandin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


In [25]:
j = 5
for k in range(j):
    i = int(random.random() * len(textos))
    print()
    print()
    print("TextoCrudo")
    print()
    print(textos["TextoCrudo"][i])
    print()
    print("TextoLimpio")
    print()
    print(textos["TextoLimpio"][i])



TextoCrudo

In Guatemala, a banana republic, the 1954 Guatemalan coup d'état ousted the left-wing President Jacobo Árbenz with material CIA support.[176] The post-Arbenz government—a military junta headed by Carlos Castillo Armas—repealed a progressive land reform law, returned nationalized property belonging to the United Fruit Company, set up a National Committee of Defense Against Communism, and decreed a Preventive Penal Law Against Communism at the request of the United States.[177]


TextoLimpio

In Guatemala a banana republic the 1954 Guatemalan coup d'état ousted the left-wing President Jacobo Árbenz with material CIA support The post-Arbenz government—a military junta headed by Carlos Castillo Armas—repealed a progressive land reform law returned nationalized property belonging to the United Fruit Company set up a National Committee of Defense Against Communism and decreed a Preventive Penal Law Against Communism at the request of the United States


TextoCrudo

In 813, Char

In [27]:
textos.to_csv("HistoriaTextosLimpios.csv")

In [28]:
textos = pd.read_csv("HistoriaTextosLimpios.csv")
textos = textos.iloc[:,1:]
textos

,TextoCrudo,TextoLimpio,TextoProcesado,Clase1,Clase2,Clase3,Clase4,Clase5,MaxFreq,Predicción,CLASE
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,Alexander III of Macedon Greek: Αλέξανδρος Γʹ ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,"During his youth, Alexander was tutored by Ari...",During his youth Alexander was tutored by Aris...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,"Alexander endeavoured to reach the ""ends of th...",Alexander endeavoured to reach the ends of the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,Alexander's legacy includes the cultural diffu...,Alexander's legacy includes the cultural diffu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,"Alexander was born in Pella, the capital of th...",Alexander was born in Pella the capital of the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...
5937,There are nevertheless a number of contemporar...,There are nevertheless a number of contemporar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
5938,Evidence suggests big-game hunter-gatherers cr...,Evidence suggests big-game hunter-gatherers cr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
5939,Hunter-gatherers would eventually flourish all...,Hunter-gatherers would eventually flourish all...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
5940,The Archaic period in the Americas saw a chang...,The Archaic period in the Americas saw a chang...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [29]:
nltk.download('stopwords')
stopwords_english = stopwords.words('english')
stop              = stopwords.words('english')
tokenizer         = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
stemmer           = PorterStemmer()    

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
for i in textos.index:
    clear_output(wait=True)        
    print(f"Tokenizando Texto Limpio {i+1} de {len(textos.index)}")
    texto_tokens = tokenizer.tokenize(textos["TextoLimpio"][i])
    texto_clean  = [word for word in texto_tokens if word not in stop and word not in string.punctuation]
    texto_stem   = [stemmer.stem(word) for word in texto_clean]
    textos["TextoProcesado"][i] = texto_stem
textos

Tokenizando Texto Limpio 5942 de 5942


,TextoCrudo,TextoLimpio,TextoProcesado,Clase1,Clase2,Clase3,Clase4,Clase5,MaxFreq,Predicción,CLASE
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,Alexander III of Macedon Greek: Αλέξανδρος Γʹ ...,"[alexand, iii, macedon, greek, αλέξανδρος, γʹ,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,"During his youth, Alexander was tutored by Ari...",During his youth Alexander was tutored by Aris...,"[youth, alexand, tutor, aristotl, age, 16, phi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,"Alexander endeavoured to reach the ""ends of th...",Alexander endeavoured to reach the ends of the...,"[alexand, endeavour, reach, end, world, great,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,Alexander's legacy includes the cultural diffu...,Alexander's legacy includes the cultural diffu...,"[alexander', legaci, includ, cultur, diffus, s...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,"Alexander was born in Pella, the capital of th...",Alexander was born in Pella the capital of the...,"[alexand, born, pella, capit, kingdom, macedon...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...
5937,There are nevertheless a number of contemporar...,There are nevertheless a number of contemporar...,"[nevertheless, number, contemporari, hunter-ga...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
5938,Evidence suggests big-game hunter-gatherers cr...,Evidence suggests big-game hunter-gatherers cr...,"[evid, suggest, big-gam, hunter-gather, cross,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
5939,Hunter-gatherers would eventually flourish all...,Hunter-gatherers would eventually flourish all...,"[hunter-gather, would, eventu, flourish, ameri...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
5940,The Archaic period in the Americas saw a chang...,The Archaic period in the Americas saw a chang...,"[archaic, period, america, saw, chang, environ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [32]:
j = 3
for k in range(j):
    i = int(random.random() * len(textos))
    print()
    print()
    print("Texto Crudo")
    print(textos["TextoCrudo"][i])
    print()
    print("Texto Limpio")
    print(textos["TextoLimpio"][i])
    print()
    print("Texto Procesado")
    print(textos["TextoProcesado"][i])



Texto Crudo
Sand dunes were much more widespread and active in many areas during the early Quaternary period. A good example is the Sand Hills region in Nebraska, USA, which covers an area of about 60,000 km2 (23,166 sq mi).[28] This region was a large, active dune field during the Pleistocene epoch, but today is largely stabilized by grass cover.[29][30]


Texto Limpio
Sand dunes were much more widespread and active in many areas during the early Quaternary period A good example is the Sand Hills region in Nebraska USA which covers an area of about 60 000 km2 23 166 sq mi This region was a large active dune field during the Pleistocene epoch but today is largely stabilized by grass cover

Texto Procesado
['sand', 'dune', 'much', 'widespread', 'activ', 'mani', 'area', 'earli', 'quaternari', 'period', 'good', 'exampl', 'sand', 'hill', 'region', 'nebraska', 'usa', 'cover', 'area', '60', '000', 'km2', '23', '166', 'sq', 'mi', 'region', 'larg', 'activ', 'dune', 'field', 'pleistocen', 'ep

In [33]:
textos.to_csv("HistoriaTextosProcesados.csv")

In [34]:
textos = pd.read_csv("HistoriaTextosProcesados.csv")
textos = textos.iloc[:,1:]
textos

,TextoCrudo,TextoLimpio,TextoProcesado,Clase1,Clase2,Clase3,Clase4,Clase5,MaxFreq,Predicción,CLASE
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,Alexander III of Macedon Greek: Αλέξανδρος Γʹ ...,"['alexand', 'iii', 'macedon', 'greek', 'αλέξαν...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,"During his youth, Alexander was tutored by Ari...",During his youth Alexander was tutored by Aris...,"['youth', 'alexand', 'tutor', 'aristotl', 'age...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,"Alexander endeavoured to reach the ""ends of th...",Alexander endeavoured to reach the ends of the...,"['alexand', 'endeavour', 'reach', 'end', 'worl...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,Alexander's legacy includes the cultural diffu...,Alexander's legacy includes the cultural diffu...,"[""alexander'"", 'legaci', 'includ', 'cultur', '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,"Alexander was born in Pella, the capital of th...",Alexander was born in Pella the capital of the...,"['alexand', 'born', 'pella', 'capit', 'kingdom...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...
5937,There are nevertheless a number of contemporar...,There are nevertheless a number of contemporar...,"['nevertheless', 'number', 'contemporari', 'hu...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
5938,Evidence suggests big-game hunter-gatherers cr...,Evidence suggests big-game hunter-gatherers cr...,"['evid', 'suggest', 'big-gam', 'hunter-gather'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
5939,Hunter-gatherers would eventually flourish all...,Hunter-gatherers would eventually flourish all...,"['hunter-gather', 'would', 'eventu', 'flourish...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
5940,The Archaic period in the Americas saw a chang...,The Archaic period in the Americas saw a chang...,"['archaic', 'period', 'america', 'saw', 'chang...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [35]:
textos["TextoProcesado"][0]

'[\'alexand\', \'iii\', \'macedon\', \'greek\', \'αλέξανδρος\', \'γʹ\', \'ὁ\', \'μακεδών\', \'aléxandro\', \'iii\', \'ho\', \'makedȏn\', \'20/21\', \'juli\', \'356bc\', \'–\', \'10/11\', \'june\', \'323bc\', \'commonli\', \'known\', \'alexand\', \'great\', \'greek\', \'ὁ\', \'μέγας\', \'ho\', \'méga\', \'king\', \'basileu\', \'ancient\', \'greek\', \'kingdom\', \'macedon\', \'member\', \'argead\', \'dynasti\', \'born\', \'pella\', \'356bc\', \'succeed\', \'father\', \'philip\', \'ii\', \'throne\', \'age\', \'20\', \'spent\', \'rule\', \'year\', \'unpreced\', \'militari\', \'campaign\', \'western\', \'asia\', \'northeast\', \'africa\', \'age\', \'thirti\', \'creat\', \'one\', \'largest\', \'empir\', \'ancient\', \'world\', \'stretch\', \'greec\', \'northwestern\', \'india\', \'undef\', \'battl\', \'wide\', \'consid\', \'one\', "history\'", \'success\', \'militari\', \'command\']'

In [36]:
textos["TextoCrudo"][0]

"Alexander III of Macedon (Greek: Αλέξανδρος Γʹ ὁ Μακεδών, Aléxandros III ho Makedȏn; 20/21 July 356 BC – 10/11 June 323 BC), commonly known as Alexander the Great (Greek: ὁ Μέγας, ho Mégas), was a king (basileus) of the ancient Greek kingdom of Macedon[a] and a member of the Argead dynasty. He was born in Pella in 356 BC and succeeded his father Philip II to the throne at the age of 20. He spent most of his ruling years on an unprecedented military campaign through western Asia and northeast Africa, and by the age of thirty, he had created one of the largest empires of the ancient world, stretching from Greece to northwestern India.[1][2] He was undefeated in battle and is widely considered one of history's most successful military commanders.[3]\n"

In [37]:
type(textos["TextoProcesado"][0])

str

In [38]:
type(textos["TextoProcesado"][0].split(","))

list

In [39]:
textos["TextoProcesado"][0].split(",")

["['alexand'",
 " 'iii'",
 " 'macedon'",
 " 'greek'",
 " 'αλέξανδρος'",
 " 'γʹ'",
 " 'ὁ'",
 " 'μακεδών'",
 " 'aléxandro'",
 " 'iii'",
 " 'ho'",
 " 'makedȏn'",
 " '20/21'",
 " 'juli'",
 " '356bc'",
 " '–'",
 " '10/11'",
 " 'june'",
 " '323bc'",
 " 'commonli'",
 " 'known'",
 " 'alexand'",
 " 'great'",
 " 'greek'",
 " 'ὁ'",
 " 'μέγας'",
 " 'ho'",
 " 'méga'",
 " 'king'",
 " 'basileu'",
 " 'ancient'",
 " 'greek'",
 " 'kingdom'",
 " 'macedon'",
 " 'member'",
 " 'argead'",
 " 'dynasti'",
 " 'born'",
 " 'pella'",
 " '356bc'",
 " 'succeed'",
 " 'father'",
 " 'philip'",
 " 'ii'",
 " 'throne'",
 " 'age'",
 " '20'",
 " 'spent'",
 " 'rule'",
 " 'year'",
 " 'unpreced'",
 " 'militari'",
 " 'campaign'",
 " 'western'",
 " 'asia'",
 " 'northeast'",
 " 'africa'",
 " 'age'",
 " 'thirti'",
 " 'creat'",
 " 'one'",
 " 'largest'",
 " 'empir'",
 " 'ancient'",
 " 'world'",
 " 'stretch'",
 " 'greec'",
 " 'northwestern'",
 " 'india'",
 " 'undef'",
 " 'battl'",
 " 'wide'",
 " 'consid'",
 " 'one'",
 ' "history\'"',

In [40]:
def limpiarWord(word):
    word = word[:-1]
    return word

def limpiarWordLeft(word):
    word = word[1:]
    return word

def extraeWords(textoProcesado):
    print(type(textoProcesado))
    words = []
    textoProcesado = textoProcesado[1:-1].split(",")
    for word in textoProcesado:
        word = word[1:]
        while word[-1:] == "'" or word[-1:] == '"' or word[-1:] == 'ʹ':
            word = limpiarWord(word)           
        while word[0] == "'" or word[0] == '"' or word[-1:] == 'ʹ':
              word = limpiarWordLeft(word)
        words.append(word)        
    return words    

In [41]:
words = extraeWords(textos["TextoProcesado"][0])
print(len(words))
print(words)
nwords = 0
for word in words:
    nwords += 1
    print(word)
print(nwords)    

<class 'str'>
78
['alexand', 'iii', 'macedon', 'greek', 'αλέξανδρος', 'γ', 'ὁ', 'μακεδών', 'aléxandro', 'iii', 'ho', 'makedȏn', '20/21', 'juli', '356bc', '–', '10/11', 'june', '323bc', 'commonli', 'known', 'alexand', 'great', 'greek', 'ὁ', 'μέγας', 'ho', 'méga', 'king', 'basileu', 'ancient', 'greek', 'kingdom', 'macedon', 'member', 'argead', 'dynasti', 'born', 'pella', '356bc', 'succeed', 'father', 'philip', 'ii', 'throne', 'age', '20', 'spent', 'rule', 'year', 'unpreced', 'militari', 'campaign', 'western', 'asia', 'northeast', 'africa', 'age', 'thirti', 'creat', 'one', 'largest', 'empir', 'ancient', 'world', 'stretch', 'greec', 'northwestern', 'india', 'undef', 'battl', 'wide', 'consid', 'one', 'history', 'success', 'militari', 'command']
alexand
iii
macedon
greek
αλέξανδρος
γ
ὁ
μακεδών
aléxandro
iii
ho
makedȏn
20/21
juli
356bc
–
10/11
june
323bc
commonli
known
alexand
great
greek
ὁ
μέγας
ho
méga
king
basileu
ancient
greek
kingdom
macedon
member
argead
dynasti
born
pella
356bc
succeed

In [42]:
frecuencias = defaultdict(int)
palabras = []
for i in textos.index:
    words = extraeWords(textos["TextoProcesado"][i])
    palabras.extend(words)
    for word in words:  
        frecuencias[(word, textos["CLASE"][i])] = frecuencias.get((word, textos["CLASE"][i]), 0) + 1    
    clear_output( wait = True )
    print(f"Creando Frecuencias {i} de {len(textos.index)}")    
vocabulario = set(palabras)    
frecuencias

Creando Frecuencias 5941 de 5942


defaultdict(int,
            {('alexand', 1): 298,
             ('iii', 1): 57,
             ('macedon', 1): 18,
             ('greek', 1): 272,
             ('αλέξανδρος', 1): 1,
             ('γ', 1): 1,
             ('ὁ', 1): 2,
             ('μακεδών', 1): 1,
             ('aléxandro', 1): 1,
             ('ho', 1): 2,
             ('makedȏn', 1): 1,
             ('20/21', 1): 1,
             ('juli', 1): 7,
             ('356bc', 1): 3,
             ('–', 1): 213,
             ('10/11', 1): 1,
             ('june', 1): 3,
             ('323bc', 1): 4,
             ('commonli', 1): 16,
             ('known', 1): 166,
             ('great', 1): 126,
             ('μέγας', 1): 1,
             ('méga', 1): 1,
             ('king', 1): 240,
             ('basileu', 1): 3,
             ('ancient', 1): 219,
             ('kingdom', 1): 128,
             ('member', 1): 28,
             ('argead', 1): 1,
             ('dynasti', 1): 83,
             ('born', 1): 15,
             ('pella', 

In [43]:
len(vocabulario)

26544

In [44]:
len(frecuencias)

46927

In [45]:
save_obj(frecuencias, "Frecuencias")
save_obj(vocabulario, "Vocabulario")

In [46]:
frecuencias = load_obj("Frecuencias")
vocabulario = load_obj("Vocabulario")

In [47]:
def etiquetas(palabras,frecuencias,clases):
    x = np.zeros((1, clases + 1))
    for palabra in palabras:
        for t in range(1, clases + 1):
            x[0,t] += frecuencias.get((palabra, t),0)
    return x

In [48]:
for i in textos.index:
    palabras = []    
    words = extraeWords(textos["TextoProcesado"][i])
    for word in words:
        assert (word in vocabulario),"La palabra " + word + " no está en el vocabulario!"     
        palabras.append(word)    
        for k in range(1, clases + 1):
            textos["Clase" + str(k)][i] = etiquetas(palabras,frecuencias,clases)[0,k]
    clear_output( wait = True )
    print(f"Creando Features {i} de {len(textos.index)}") 
textos

Creando Features 5941 de 5942


,TextoCrudo,TextoLimpio,TextoProcesado,Clase1,Clase2,Clase3,Clase4,Clase5,MaxFreq,Predicción,CLASE
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,Alexander III of Macedon Greek: Αλέξανδρος Γʹ ...,"['alexand', 'iii', 'macedon', 'greek', 'αλέξαν...",5765.0,4813.0,4377.0,2796.0,3298.0,NaN,NaN,1
1,"During his youth, Alexander was tutored by Ari...",During his youth Alexander was tutored by Aris...,"['youth', 'alexand', 'tutor', 'aristotl', 'age...",5726.0,5416.0,4039.0,3072.0,2520.0,NaN,NaN,1
2,"Alexander endeavoured to reach the ""ends of th...",Alexander endeavoured to reach the ends of the...,"['alexand', 'endeavour', 'reach', 'end', 'worl...",3986.0,3825.0,3753.0,4337.0,2094.0,NaN,NaN,1
3,Alexander's legacy includes the cultural diffu...,Alexander's legacy includes the cultural diffu...,"[""alexander'"", 'legaci', 'includ', 'cultur', '...",7024.0,6021.0,3812.0,3304.0,4695.0,NaN,NaN,1
4,"Alexander was born in Pella, the capital of th...",Alexander was born in Pella the capital of the...,"['alexand', 'born', 'pella', 'capit', 'kingdom...",3105.0,2595.0,2873.0,1062.0,1217.0,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...
5937,There are nevertheless a number of contemporar...,There are nevertheless a number of contemporar...,"['nevertheless', 'number', 'contemporari', 'hu...",3415.0,3850.0,3386.0,2612.0,4557.0,NaN,NaN,5
5938,Evidence suggests big-game hunter-gatherers cr...,Evidence suggests big-game hunter-gatherers cr...,"['evid', 'suggest', 'big-gam', 'hunter-gather'...",2355.0,2270.0,2916.0,2085.0,6659.0,NaN,NaN,5
5939,Hunter-gatherers would eventually flourish all...,Hunter-gatherers would eventually flourish all...,"['hunter-gather', 'would', 'eventu', 'flourish...",2534.0,2625.0,2666.0,2598.0,5156.0,NaN,NaN,5
5940,The Archaic period in the Americas saw a chang...,The Archaic period in the Americas saw a chang...,"['archaic', 'period', 'america', 'saw', 'chang...",2173.0,2227.0,2065.0,1861.0,4283.0,NaN,NaN,5


In [49]:
textos.to_csv("Historia.csv")

In [50]:
textos = pd.read_csv("Historia.csv")
textos = textos.iloc[:,1:]
textos

,TextoCrudo,TextoLimpio,TextoProcesado,Clase1,Clase2,Clase3,Clase4,Clase5,MaxFreq,Predicción,CLASE
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,Alexander III of Macedon Greek: Αλέξανδρος Γʹ ...,"['alexand', 'iii', 'macedon', 'greek', 'αλέξαν...",5765.0,4813.0,4377.0,2796.0,3298.0,NaN,NaN,1
1,"During his youth, Alexander was tutored by Ari...",During his youth Alexander was tutored by Aris...,"['youth', 'alexand', 'tutor', 'aristotl', 'age...",5726.0,5416.0,4039.0,3072.0,2520.0,NaN,NaN,1
2,"Alexander endeavoured to reach the ""ends of th...",Alexander endeavoured to reach the ends of the...,"['alexand', 'endeavour', 'reach', 'end', 'worl...",3986.0,3825.0,3753.0,4337.0,2094.0,NaN,NaN,1
3,Alexander's legacy includes the cultural diffu...,Alexander's legacy includes the cultural diffu...,"[""alexander'"", 'legaci', 'includ', 'cultur', '...",7024.0,6021.0,3812.0,3304.0,4695.0,NaN,NaN,1
4,"Alexander was born in Pella, the capital of th...",Alexander was born in Pella the capital of the...,"['alexand', 'born', 'pella', 'capit', 'kingdom...",3105.0,2595.0,2873.0,1062.0,1217.0,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...
5937,There are nevertheless a number of contemporar...,There are nevertheless a number of contemporar...,"['nevertheless', 'number', 'contemporari', 'hu...",3415.0,3850.0,3386.0,2612.0,4557.0,NaN,NaN,5
5938,Evidence suggests big-game hunter-gatherers cr...,Evidence suggests big-game hunter-gatherers cr...,"['evid', 'suggest', 'big-gam', 'hunter-gather'...",2355.0,2270.0,2916.0,2085.0,6659.0,NaN,NaN,5
5939,Hunter-gatherers would eventually flourish all...,Hunter-gatherers would eventually flourish all...,"['hunter-gather', 'would', 'eventu', 'flourish...",2534.0,2625.0,2666.0,2598.0,5156.0,NaN,NaN,5
5940,The Archaic period in the Americas saw a chang...,The Archaic period in the Americas saw a chang...,"['archaic', 'period', 'america', 'saw', 'chang...",2173.0,2227.0,2065.0,1861.0,4283.0,NaN,NaN,5


In [51]:
for i in textos.index:
    print(textos.loc[i,"CLASE"] ==  np.argmax(textos.iloc[i,columna:columna + clases].values) + 1)
    textos["MaxFreq"][i] = textos.loc[i,"CLASE"] ==  np.argmax(textos.iloc[i,columna:columna + clases].values) + 1
    clear_output( wait = True )
    print(f"Comparando CLASE y features {i} de {len(textos.index)}") 
textos      

Comparando CLASE y features 5941 de 5942


,TextoCrudo,TextoLimpio,TextoProcesado,Clase1,Clase2,Clase3,Clase4,Clase5,MaxFreq,Predicción,CLASE
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,Alexander III of Macedon Greek: Αλέξανδρος Γʹ ...,"['alexand', 'iii', 'macedon', 'greek', 'αλέξαν...",5765.0,4813.0,4377.0,2796.0,3298.0,True,NaN,1
1,"During his youth, Alexander was tutored by Ari...",During his youth Alexander was tutored by Aris...,"['youth', 'alexand', 'tutor', 'aristotl', 'age...",5726.0,5416.0,4039.0,3072.0,2520.0,True,NaN,1
2,"Alexander endeavoured to reach the ""ends of th...",Alexander endeavoured to reach the ends of the...,"['alexand', 'endeavour', 'reach', 'end', 'worl...",3986.0,3825.0,3753.0,4337.0,2094.0,False,NaN,1
3,Alexander's legacy includes the cultural diffu...,Alexander's legacy includes the cultural diffu...,"[""alexander'"", 'legaci', 'includ', 'cultur', '...",7024.0,6021.0,3812.0,3304.0,4695.0,True,NaN,1
4,"Alexander was born in Pella, the capital of th...",Alexander was born in Pella the capital of the...,"['alexand', 'born', 'pella', 'capit', 'kingdom...",3105.0,2595.0,2873.0,1062.0,1217.0,True,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...
5937,There are nevertheless a number of contemporar...,There are nevertheless a number of contemporar...,"['nevertheless', 'number', 'contemporari', 'hu...",3415.0,3850.0,3386.0,2612.0,4557.0,True,NaN,5
5938,Evidence suggests big-game hunter-gatherers cr...,Evidence suggests big-game hunter-gatherers cr...,"['evid', 'suggest', 'big-gam', 'hunter-gather'...",2355.0,2270.0,2916.0,2085.0,6659.0,True,NaN,5
5939,Hunter-gatherers would eventually flourish all...,Hunter-gatherers would eventually flourish all...,"['hunter-gather', 'would', 'eventu', 'flourish...",2534.0,2625.0,2666.0,2598.0,5156.0,True,NaN,5
5940,The Archaic period in the Americas saw a chang...,The Archaic period in the Americas saw a chang...,"['archaic', 'period', 'america', 'saw', 'chang...",2173.0,2227.0,2065.0,1861.0,4283.0,True,NaN,5


In [52]:
textos.to_csv("Historia.csv")

In [53]:
textos = pd.read_csv("Historia.csv")
textos = textos.iloc[:,1:]
textos

,TextoCrudo,TextoLimpio,TextoProcesado,Clase1,Clase2,Clase3,Clase4,Clase5,MaxFreq,Predicción,CLASE
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,Alexander III of Macedon Greek: Αλέξανδρος Γʹ ...,"['alexand', 'iii', 'macedon', 'greek', 'αλέξαν...",5765.0,4813.0,4377.0,2796.0,3298.0,True,NaN,1
1,"During his youth, Alexander was tutored by Ari...",During his youth Alexander was tutored by Aris...,"['youth', 'alexand', 'tutor', 'aristotl', 'age...",5726.0,5416.0,4039.0,3072.0,2520.0,True,NaN,1
2,"Alexander endeavoured to reach the ""ends of th...",Alexander endeavoured to reach the ends of the...,"['alexand', 'endeavour', 'reach', 'end', 'worl...",3986.0,3825.0,3753.0,4337.0,2094.0,False,NaN,1
3,Alexander's legacy includes the cultural diffu...,Alexander's legacy includes the cultural diffu...,"[""alexander'"", 'legaci', 'includ', 'cultur', '...",7024.0,6021.0,3812.0,3304.0,4695.0,True,NaN,1
4,"Alexander was born in Pella, the capital of th...",Alexander was born in Pella the capital of the...,"['alexand', 'born', 'pella', 'capit', 'kingdom...",3105.0,2595.0,2873.0,1062.0,1217.0,True,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...
5937,There are nevertheless a number of contemporar...,There are nevertheless a number of contemporar...,"['nevertheless', 'number', 'contemporari', 'hu...",3415.0,3850.0,3386.0,2612.0,4557.0,True,NaN,5
5938,Evidence suggests big-game hunter-gatherers cr...,Evidence suggests big-game hunter-gatherers cr...,"['evid', 'suggest', 'big-gam', 'hunter-gather'...",2355.0,2270.0,2916.0,2085.0,6659.0,True,NaN,5
5939,Hunter-gatherers would eventually flourish all...,Hunter-gatherers would eventually flourish all...,"['hunter-gather', 'would', 'eventu', 'flourish...",2534.0,2625.0,2666.0,2598.0,5156.0,True,NaN,5
5940,The Archaic period in the Americas saw a chang...,The Archaic period in the Americas saw a chang...,"['archaic', 'period', 'america', 'saw', 'chang...",2173.0,2227.0,2065.0,1861.0,4283.0,True,NaN,5


In [54]:
textos.groupby(by=["MaxFreq"]).count()

,TextoCrudo,TextoLimpio,TextoProcesado,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
MaxFreq,,,,,,,,,,
False,1085,1085,1085,1085,1085,1085,1085,1085,0,1085
True,4857,4857,4857,4857,4857,4857,4857,4857,0,4857


In [55]:
rate = 100 * 4857 / (1085 + 4857)
print("Porcentaje de features acertados",rate)

Porcentaje de features acertados 81.74015483002356


**Vamos a procesar estos datos en una red neuronal**

**Obtuvimos un accuracy del 85% con el modelo neuronal**

In [56]:
textos = pd.read_csv("Historia.csv")
textos = textos.iloc[:,1:]
textos

,TextoCrudo,TextoLimpio,TextoProcesado,Clase1,Clase2,Clase3,Clase4,Clase5,MaxFreq,Predicción,CLASE
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,Alexander III of Macedon Greek: Αλέξανδρος Γʹ ...,"['alexand', 'iii', 'macedon', 'greek', 'αλέξαν...",5765.0,4813.0,4377.0,2796.0,3298.0,True,1.0,1
1,"During his youth, Alexander was tutored by Ari...",During his youth Alexander was tutored by Aris...,"['youth', 'alexand', 'tutor', 'aristotl', 'age...",5726.0,5416.0,4039.0,3072.0,2520.0,True,1.0,1
2,"Alexander endeavoured to reach the ""ends of th...",Alexander endeavoured to reach the ends of the...,"['alexand', 'endeavour', 'reach', 'end', 'worl...",3986.0,3825.0,3753.0,4337.0,2094.0,False,1.0,1
3,Alexander's legacy includes the cultural diffu...,Alexander's legacy includes the cultural diffu...,"[""alexander'"", 'legaci', 'includ', 'cultur', '...",7024.0,6021.0,3813.0,3304.0,4695.0,True,1.0,1
4,"Alexander was born in Pella, the capital of th...",Alexander was born in Pella the capital of the...,"['alexand', 'born', 'pella', 'capit', 'kingdom...",3105.0,2595.0,2874.0,1062.0,1217.0,True,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...
5937,There are nevertheless a number of contemporar...,There are nevertheless a number of contemporar...,"['nevertheless', 'number', 'contemporari', 'hu...",3415.0,3850.0,3387.0,2612.0,4557.0,True,5.0,5
5938,Evidence suggests big-game hunter-gatherers cr...,Evidence suggests big-game hunter-gatherers cr...,"['evid', 'suggest', 'big-gam', 'hunter-gather'...",2355.0,2270.0,2917.0,2085.0,6659.0,True,5.0,5
5939,Hunter-gatherers would eventually flourish all...,Hunter-gatherers would eventually flourish all...,"['hunter-gather', 'would', 'eventu', 'flourish...",2534.0,2625.0,2667.0,2598.0,5156.0,True,5.0,5
5940,The Archaic period in the Americas saw a chang...,The Archaic period in the Americas saw a chang...,"['archaic', 'period', 'america', 'saw', 'chang...",2173.0,2227.0,2068.0,1861.0,4283.0,True,5.0,5


In [ ]:
textos.sample(30)

In [57]:
pifiadas = textos[textos["Predicción"] != textos["CLASE"]]
pifiadas

,TextoCrudo,TextoLimpio,TextoProcesado,Clase1,Clase2,Clase3,Clase4,Clase5,MaxFreq,Predicción,CLASE
9,"When Alexander was 13, Philip began to search ...",When Alexander was 13 Philip began to search f...,"['alexand', '13', 'philip', 'began', 'search',...",1615.0,919.0,2195.0,751.0,414.0,False,3.0,1
15,"Philip and his army joined his son in 338 BC, ...",Philip and his army joined his son in 338BC an...,"['philip', 'armi', 'join', 'son', '338bc', 'ma...",1879.0,1939.0,3226.0,1998.0,456.0,False,3.0,1
16,"As Philip marched south, his opponents blocked...",As Philip marched south his opponents blocked ...,"['philip', 'march', 'south', 'oppon', 'block',...",3754.0,2777.0,4910.0,2889.0,1755.0,False,3.0,1
17,"After the victory at Chaeronea, Philip and Ale...",After the victory at Chaeronea Philip and Alex...,"['victori', 'chaeronea', 'philip', 'alexand', ...",3372.0,2706.0,3779.0,3239.0,1487.0,False,3.0,1
18,"When Philip returned to Pella, he fell in love...",When Philip returned to Pella he fell in love ...,"['philip', 'return', 'pella', 'fell', 'love', ...",1752.0,1400.0,1949.0,1198.0,711.0,False,3.0,1
...,...,...,...,...,...,...,...,...,...,...,...
5895,"""The material they call Aboriginal art is almo...",The material they call Aboriginal art is almos...,"['materi', 'call', 'aborigin', 'art', 'almost'...",1296.0,1074.0,1248.0,833.0,1616.0,True,1.0,5
5896,The inquiry's final report[60] made recommenda...,The inquiry's final report made recommendation...,"[""inquiry'"", 'final', 'report', 'made', 'recom...",541.0,530.0,623.0,675.0,583.0,False,3.0,5
5900,"The phrase ""traditional cultural expressions"" ...",The phrase traditional cultural expressions is...,"['phrase', 'tradit', 'cultur', 'express', 'use...",1913.0,1611.0,1365.0,1030.0,2384.0,True,1.0,5
5902,"The Museum for Australian Aboriginal art ""La g...",The Museum for Australian Aboriginal art La gr...,"['museum', 'australian', 'aborigin', 'art', 'l...",1468.0,1486.0,1613.0,921.0,1991.0,True,3.0,5


In [58]:
pifiadas.groupby(by=["CLASE"]).count()

,TextoCrudo,TextoLimpio,TextoProcesado,Clase1,Clase2,Clase3,Clase4,Clase5,MaxFreq,Predicción
CLASE,,,,,,,,,,
1,151,151,151,151,151,151,151,151,151,151
2,184,184,184,184,184,184,184,184,184,184
3,258,258,258,258,258,258,258,258,258,258
4,177,177,177,177,177,177,177,177,177,177
5,95,95,95,95,95,95,95,95,95,95


In [59]:
cont = 0
cantTrues = 0
for i in pifiadas.index:
    if pifiadas["Predicción"][i]  ==  (np.argmax(pifiadas.iloc[cont,columna:columna + clases]) + 1):
       cantTrues +=1                               
    cont += 1
print("Cantidad de veces que la predicción coincide con la clase con la máxima frecuencia:",cantTrues)
print("Cantidad de total de predicciones pifiadas:",cont)
print("Rate Freq/Pifios:", int(100 * cantTrues/cont)) 

Cantidad de veces que la predicción coincide con la clase con la máxima frecuencia: 626
Cantidad de total de predicciones pifiadas: 865
Rate Freq/Pifios: 72


In [60]:
aciertos = textos[textos["Predicción"] == textos["CLASE"]]
aciertos

,TextoCrudo,TextoLimpio,TextoProcesado,Clase1,Clase2,Clase3,Clase4,Clase5,MaxFreq,Predicción,CLASE
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,Alexander III of Macedon Greek: Αλέξανδρος Γʹ ...,"['alexand', 'iii', 'macedon', 'greek', 'αλέξαν...",5765.0,4813.0,4377.0,2796.0,3298.0,True,1.0,1
1,"During his youth, Alexander was tutored by Ari...",During his youth Alexander was tutored by Aris...,"['youth', 'alexand', 'tutor', 'aristotl', 'age...",5726.0,5416.0,4039.0,3072.0,2520.0,True,1.0,1
2,"Alexander endeavoured to reach the ""ends of th...",Alexander endeavoured to reach the ends of the...,"['alexand', 'endeavour', 'reach', 'end', 'worl...",3986.0,3825.0,3753.0,4337.0,2094.0,False,1.0,1
3,Alexander's legacy includes the cultural diffu...,Alexander's legacy includes the cultural diffu...,"[""alexander'"", 'legaci', 'includ', 'cultur', '...",7024.0,6021.0,3813.0,3304.0,4695.0,True,1.0,1
4,"Alexander was born in Pella, the capital of th...",Alexander was born in Pella the capital of the...,"['alexand', 'born', 'pella', 'capit', 'kingdom...",3105.0,2595.0,2874.0,1062.0,1217.0,True,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...
5937,There are nevertheless a number of contemporar...,There are nevertheless a number of contemporar...,"['nevertheless', 'number', 'contemporari', 'hu...",3415.0,3850.0,3387.0,2612.0,4557.0,True,5.0,5
5938,Evidence suggests big-game hunter-gatherers cr...,Evidence suggests big-game hunter-gatherers cr...,"['evid', 'suggest', 'big-gam', 'hunter-gather'...",2355.0,2270.0,2917.0,2085.0,6659.0,True,5.0,5
5939,Hunter-gatherers would eventually flourish all...,Hunter-gatherers would eventually flourish all...,"['hunter-gather', 'would', 'eventu', 'flourish...",2534.0,2625.0,2667.0,2598.0,5156.0,True,5.0,5
5940,The Archaic period in the Americas saw a chang...,The Archaic period in the Americas saw a chang...,"['archaic', 'period', 'america', 'saw', 'chang...",2173.0,2227.0,2068.0,1861.0,4283.0,True,5.0,5


In [61]:
cont = 0
cantTrues = 0
for i in aciertos.index:
    if aciertos["Predicción"][i]  ==  (np.argmax(aciertos.iloc[cont,columna:columna + clases]) + 1):
       cantTrues +=1                               
    cont += 1
print("Cantidad de veces que la predicción coincide con la clase con la máxima frecuencia:",cantTrues)
print("Cantidad de total de predicciones acertadas:",cont)
print("Rate Freq/Aciertos:", int(100 * cantTrues/cont)) 

Cantidad de veces que la predicción coincide con la clase con la máxima frecuencia: 4700
Cantidad de total de predicciones acertadas: 5077
Rate Freq/Aciertos: 92


In [62]:
pifiadas

,TextoCrudo,TextoLimpio,TextoProcesado,Clase1,Clase2,Clase3,Clase4,Clase5,MaxFreq,Predicción,CLASE
9,"When Alexander was 13, Philip began to search ...",When Alexander was 13 Philip began to search f...,"['alexand', '13', 'philip', 'began', 'search',...",1615.0,919.0,2195.0,751.0,414.0,False,3.0,1
15,"Philip and his army joined his son in 338 BC, ...",Philip and his army joined his son in 338BC an...,"['philip', 'armi', 'join', 'son', '338bc', 'ma...",1879.0,1939.0,3226.0,1998.0,456.0,False,3.0,1
16,"As Philip marched south, his opponents blocked...",As Philip marched south his opponents blocked ...,"['philip', 'march', 'south', 'oppon', 'block',...",3754.0,2777.0,4910.0,2889.0,1755.0,False,3.0,1
17,"After the victory at Chaeronea, Philip and Ale...",After the victory at Chaeronea Philip and Alex...,"['victori', 'chaeronea', 'philip', 'alexand', ...",3372.0,2706.0,3779.0,3239.0,1487.0,False,3.0,1
18,"When Philip returned to Pella, he fell in love...",When Philip returned to Pella he fell in love ...,"['philip', 'return', 'pella', 'fell', 'love', ...",1752.0,1400.0,1949.0,1198.0,711.0,False,3.0,1
...,...,...,...,...,...,...,...,...,...,...,...
5895,"""The material they call Aboriginal art is almo...",The material they call Aboriginal art is almos...,"['materi', 'call', 'aborigin', 'art', 'almost'...",1296.0,1074.0,1248.0,833.0,1616.0,True,1.0,5
5896,The inquiry's final report[60] made recommenda...,The inquiry's final report made recommendation...,"[""inquiry'"", 'final', 'report', 'made', 'recom...",541.0,530.0,623.0,675.0,583.0,False,3.0,5
5900,"The phrase ""traditional cultural expressions"" ...",The phrase traditional cultural expressions is...,"['phrase', 'tradit', 'cultur', 'express', 'use...",1913.0,1611.0,1365.0,1030.0,2384.0,True,1.0,5
5902,"The Museum for Australian Aboriginal art ""La g...",The Museum for Australian Aboriginal art La gr...,"['museum', 'australian', 'aborigin', 'art', 'l...",1468.0,1486.0,1613.0,921.0,1991.0,True,3.0,5


In [63]:
textos.sample(10)

,TextoCrudo,TextoLimpio,TextoProcesado,Clase1,Clase2,Clase3,Clase4,Clase5,MaxFreq,Predicción,CLASE
281,"In 50 BC, the Senate (led by Pompey) ordered C...",In 50BC the Senate led by Pompey ordered Caesa...,"['50bc', 'senat', 'led', 'pompey', 'order', 'c...",6638.0,3377.0,3287.0,3530.0,1746.0,True,1.0,1
205,Pilum (spear): The Roman heavy spear was a wea...,Pilum spear : The Roman heavy spear was a weap...,"['pilum', 'spear', 'roman', 'heavi', 'spear', ...",4356.0,2554.0,1790.0,1543.0,2331.0,True,1.0,1
974,The earliest enemies of the Parthians were the...,The earliest enemies of the Parthians were the...,"['earliest', 'enemi', 'parthian', 'seleucid', ...",13975.0,9557.0,6844.0,6658.0,3677.0,True,1.0,1
2875,The day after Anne's execution in 1536 the 45-...,The day after Anne's execution in 1536 the 45-...,"['day', ""anne'"", 'execut', '1536', '45', 'year...",2079.0,3190.0,4536.0,1863.0,1508.0,True,3.0,3
506,"By about 5500 BC, small tribes living in the N...",By about 5500BC small tribes living in the Nil...,"['5500bc', 'small', 'tribe', 'live', 'nile', '...",2521.0,1972.0,1258.0,995.0,4458.0,False,5.0,1
80,When faced with opponents who used unfamiliar ...,When faced with opponents who used unfamiliar ...,"['face', 'oppon', 'use', 'unfamiliar', 'fight'...",2441.0,1477.0,1503.0,1569.0,2217.0,True,1.0,1
5222,The Savanna Pastoral Neolithic or SPN (formerl...,The Savanna Pastoral Neolithic or SPN formerly...,"['savanna', 'pastor', 'neolith', 'spn', 'forme...",3248.0,2663.0,2095.0,1351.0,6444.0,True,5.0,5
874,"Periodically, rulers signed ""clean slate"" decr...",Periodically rulers signed clean slate decrees...,"['period', 'ruler', 'sign', 'clean', 'slate', ...",3987.0,3835.0,3842.0,2892.0,3536.0,True,1.0,1
643,"At this time, Marius began his quarrel with Su...",At this time Marius began his quarrel with Sul...,"['time', 'mariu', 'began', 'quarrel', 'sulla',...",2481.0,1746.0,1572.0,1625.0,1027.0,True,1.0,1
1994,"The dynastic name, medieval Latin Merovingi or...",The dynastic name medieval Latin Merovingi or ...,"['dynast', 'name', 'mediev', 'latin', 'merovin...",1440.0,2434.0,1584.0,843.0,1121.0,True,2.0,2


In [ ]:
# frecuenciasMaximas: diccionario que relaciona cada palabra con la clase
# que tiene la máxima frecuencia de aparición de esa palabra

frecuenciasMaximas = {} 

for word in vocabulario:    
    freqWord = []
    for k in range(clases + 1):
        freqWord.append(0)   
        
    for k in range(1, len(freqWord)):
        freqWord[k] = frecuencias.get((word, k),0)    
    for k in range(1, len(freqWord)):
        frecuenciasMaximas[word] = np.argmax(freqWord)   


print()    
print("Frecuencias máximas")
for key in frecuenciasMaximas.keys():
    print(key,frecuenciasMaximas[key])    

clasesPalabrasFM = {}
for k in range(1,clases + 1):
    clasesPalabrasFM[k] = []     
for word in frecuenciasMaximas.keys():
    clasesPalabrasFM[frecuenciasMaximas[word]].append(word)
    
totV = 0    

# Verificamos que todas las palabras del vocabulario estén en el dicciconario de 
# frecuencias de clases

for k in range(1, clases + 1):
    totV += len(clasesPalabrasFM[k])  
assert(totV == len(vocabulario)),"PALABRAS FALTANTES"    

In [ ]:
frecuenciasMaxDetalle = {}     
    
for word, value in frecuenciasMaximas.items():
    freqWord = []
    for k in range(clases + 1):
        freqWord.append(0)     
    for k in range(1, len(freqWord)):
        freqWord[k] = frecuencias.get((word, k),0)        
    
    if sum(freqWord) == frecuencias[word, np.argmax(freqWord)]:
        rate = 99
    else:    
        rate = int(100 * frecuencias[word, np.argmax(freqWord)] / sum(freqWord))
    
    frecuenciasMaxDetalle[word] = value, rate - 19
    
print()
print("frecuenciasMaxDetalle")
print()
cont = 0
nconfusas = 0
confusas = []
for key in frecuenciasMaxDetalle.keys():
    cont +=1
    if cont > 100000:
        break
    if frecuenciasMaxDetalle[key][1] < 60:    
        print("PALABRA CONFUSA: ",key, frecuenciasMaxDetalle[key])   
        print(frecuenciasMaximas[key])
        freqWord = []
        for k in range(clases + 1):
            freqWord.append(0)   
        for k in range(1, len(freqWord)):
            freqWord[k] = frecuencias.get((key, k),0)
        
        # print("Mínimo",np.min(freqWord[1:]))
        # print("Suma",sum(freqWord[1:]))
        print()
        rateMinSum =  int(500 *  np.min(freqWord[1:]) / sum(freqWord[1:]))
        if rateMinSum > 25:
            nconfusas += 1
            print(freqWord[1:])
            print("minimo",np.min(freqWord[1:])  )
            print(sum(freqWord[1:]))
            print(freqWord[frecuenciasMaxDetalle[key][0]])
            print("% Mínimo / suma:", int(500 *  np.min(freqWord[1:]) / sum(freqWord[1:]) ) )
            # print("key",type(key),key)
            # print("key.shape",key.split().shape)
            confusas.append(key)
        
print("cantidad de palabras confusas = ",nconfusas)   
len(confusas)
confusas
print("cantidad de palabras confusas = ",nconfusas)   
len(confusas)

In [ ]:
len(vocabulario)

In [ ]:
len(confusas)

In [ ]:
textos["TextoProcesado"][0]

In [ ]:
textos

## Para extraer las palabras confusas debemos hacerlo sobre el dataset total, con y sin pifiadas  

In [ ]:
conta = 0
for i in textos.index:
    conta += 1
    if conta > 222222:
        break   
    lista = []
    words = extraeWords(textos["TextoProcesado"][i])
    for word in words:
        assert (word in vocabulario),"La palabra " + word + " no está en el vocabulario!"    
        if word not in confusas:
            lista.append(word) 
    clear_output( wait = True )
    print(f"Procesando {i} de {len(textos.index)}") 
    textos["TextoProcesado"][i] = lista
textos

## Ahora que el texto ha quedado sin las palabras confusas vamos a volver a extraer los features  

In [ ]:
for i in textos.index:
    print(i)
    palabras = []    
    # words = extraeWords(textos["TextoProcesado"][i])
    words = textos["TextoProcesado"][i]
    for word in words:
        assert (word in vocabulario),"La palabra " + word + " no está en el vocabulario!"     
        palabras.append(word)    
        for k in range(1, clases + 1):
            textos["Clase" + str(k)][i] = etiquetas(palabras,frecuencias,clases)[0,k]
    clear_output( wait = True )
    print(f"Creando Features {i} de {len(textos.index)}") 
textos

In [ ]:
for i in textos.index:
    print(textos.loc[i,"CLASE"] ==  np.argmax(textos.iloc[i,columna:columna + clases].values) + 1)
    textos["MaxFreq"][i] = textos.loc[i,"CLASE"] ==  np.argmax(textos.iloc[i,columna:columna + clases].values) + 1
    clear_output( wait = True )
    print(f"Comparando CLASE y features {i} de {len(textos.index)}") 
textos      

In [ ]:
textos.groupby(by=["MaxFreq"]).count()

In [ ]:
rate = 100 * 4937 / (1005 + 4937)
print("Porcentaje de features acertados",rate)

In [ ]:
textos.to_csv("Historia.csv")

In [ ]:
textos = pd.read_csv("Historia.csv")
textos = textos.iloc[:,1:]
textos

**Vamos al modelo neuronal**

In [ ]:
save_obj(vocabulario, "Vocabulario")
save_obj(frecuencias, "Frecuencias")
save_obj(confusas, "Confusas")

**Logramos un accuracy del 85%**